In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/salary-dataset/Salary_Data.csv


In [2]:
df=pd.read_csv('/kaggle/input/salary-dataset/Salary_Data.csv')

In [3]:
df.head()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0
1,28.0,Female,Master's,Data Analyst,3.0,65000.0
2,45.0,Male,PhD,Senior Manager,15.0,150000.0
3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0
4,52.0,Male,Master's,Director,20.0,200000.0


In [4]:
df=df.drop('Job Title', axis=1)

In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
le = LabelEncoder()

In [7]:
df['Gender']=le.fit_transform(df['Gender'])

In [8]:
df.head()

,Age,Gender,Education Level,Years of Experience,Salary
0,32.0,1,Bachelor's,5.0,90000.0
1,28.0,0,Master's,3.0,65000.0
2,45.0,1,PhD,15.0,150000.0
3,36.0,0,Bachelor's,7.0,60000.0
4,52.0,1,Master's,20.0,200000.0


In [9]:
    from sklearn.preprocessing import OrdinalEncoder

In [10]:
df['Education Level'].unique()

array(["Bachelor's", "Master's", 'PhD', nan, "Bachelor's Degree",
       "Master's Degree", 'High School', 'phD'], dtype=object)

In [11]:
df['Education Level'] = df['Education Level'].replace({
    "Bachelor's Degree": "Bachelor's",
    "Master's Degree": "Master's",
    "phD": "PhD",
    "phd": "PhD",
    "PHD": "PhD"
})
df['Education Level'].unique()

array(["Bachelor's", "Master's", 'PhD', nan, 'High School'], dtype=object)

In [12]:
categories = [[np.nan, 'High School', "Bachelor's", "Master's", 'PhD']]
oe_ordered = OrdinalEncoder(categories=categories)
encoded = oe_ordered.fit_transform(df[['Education Level']])

# Combine encoded values with original data for clarity
df['Education Level'] = encoded
df=df.dropna()
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 6698 entries, 0 to 6703
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Age                  6698 non-null   float64
 1   Gender               6698 non-null   int64  
 2   Education Level      6698 non-null   float64
 3   Years of Experience  6698 non-null   float64
 4   Salary               6698 non-null   float64
dtypes: float64(4), int64(1)
memory usage: 314.0 KB


In [13]:
from sklearn.preprocessing import StandardScaler

In [14]:
scaler=StandardScaler()

In [15]:
X = df.drop('Salary', axis=1)
y = df['Salary']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(df)

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [17]:
lr=LinearRegression()

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
lr.fit(X_train,y_train)

LinearRegression()

In [20]:
y_pred=lr.predict(X_test)

In [21]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Predictions
y_pred = lr.predict(X_test)

# Evaluation metrics
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("R² Score:", r2)
print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)

R² Score: 0.7078827917191846
Mean Absolute Error: 22477.452344367037
Mean Squared Error: 832960315.2147213


In [22]:
from sklearn.decomposition import PCA

In [23]:


# 1️⃣ Fit PCA on training set only
pca = PCA(n_components=2)
X_pca_train = pca.fit_transform(X_train)

# 2️⃣ Transform test set using the same PCA
X_pca_test = pca.transform(X_test)

# 3️⃣ Train regression model on PCA features
lr = LinearRegression()
lr.fit(X_pca_train, y_train)

# 4️⃣ Make predictions
y_pred = lr.predict(X_pca_test)

# 5️⃣ Evaluate RMSE, MAE, R²
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("RMSE:", rmse)
print("MAE:", mae)
print("R² Score:", r2)


RMSE: 30827.678752091455
MAE: 24455.488156659674
R² Score: 0.6667159884107137
